# Atividade: CNNs para Classificação

Neste notebook, iremos preparar nosso próprio dataset e treinar um modelo de classificação de imagens.

## Preparando os dados

Os dados desta atividade serão baixados da internet. Utilizaremos para isso buscadores comuns. Em seguida, dividiremos em treinamento e validação.

In [95]:
%pip install icrawler --quiet
%pip install pyimagedl --quiet
%pip install seaborn --quiet



Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/util

KeyboardInterrupt: 

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import shutil
import random
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def set_seed(seed=42):
    """Congela a aleatoriedade para tornar o experimento reprodutível."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configuração de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

In [ ]:
import random
from google.colab import drive
#from icrawler.builtin import GoogleImageCrawler, BingImageCrawler

### Adquirindo as Imagens

Utilizaremos o iCrawler para baixar imagens em buscadores através de termos especificados. Defina sua lista de classes.

In [ ]:
# get images either from Drive or local storage
use_local = False

In [ ]:
from imagedl import imagedl

def download_images_generic(keyword, label, output_dir="data", target_total=100):
    final_folder = os.path.join(output_dir, label)
    os.makedirs(final_folder, exist_ok=True)

    # Conta imagens já existentes
    count_existing = len([
        f for f in os.listdir(final_folder)
        if f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp'))
    ])

    if count_existing >= target_total:
        print(f"✅ [{label}] Já possui {count_existing} imagens. Pulando...")
        return

    needed = target_total - count_existing
    print(f"⬇️ [{label}] Possui {count_existing}. Baixando mais {needed} imagens...")

    temp_folder = final_folder + "_tmp"
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)

    try:
        # Configuração do cliente
        client = imagedl.ImageClient(
            image_source='GoogleImageClient',
            init_image_client_cfg={'work_dir': temp_folder},
            # Baixamos um pouco a mais para garantir caso alguns falhem
            search_limits=needed + 30,
            num_threadings=4,
        )

        # Busca e download
        image_infos = client.search(keyword, search_limits_overrides=needed + 30)
        client.download(image_infos=image_infos)

        moved = 0
        new_count = count_existing

        # Move e renomeia as imagens do temp para a pasta final
        for root, dirs, files in os.walk(temp_folder):
            for file in files:
                if moved >= needed:
                    break

                if file.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
                    src = os.path.join(root, file)
                    # Formato: mondstadt_0001.jpg, liyue_0050.jpg, etc.
                    dst = os.path.join(final_folder, f"{label}_{new_count:04d}.jpg")
                    try:
                        shutil.move(src, dst)
                        moved += 1
                        new_count += 1
                    except:
                        pass

        print(f"Concluído! Agora [{label}] tem {new_count} imagens.")

    except Exception as e:
        print(f"Erro ao baixar imagens para {label}: {e}")
    finally:
        if os.path.exists(temp_folder):
            shutil.rmtree(temp_folder)

In [ ]:
# -------------------------------------------------------------
# CONFIGURAÇÃO PARA O PROJETO GENSHIN
# -------------------------------------------------------------
OUTPUT_DIR = "/content/drive/MyDrive/genshin_landscape_dataset"

# Estratégia: Definimos as sub-regiões visuais distintas aqui.
# O modelo vai aprender "Enkanomiya", "Dragonspine", etc.
classes = {
    # MONDTADT
    "mondstadt": "genshin impact mondstadt landscape scenery -character -build",
    "dragonspine": "genshin impact dragonspine landscape scenery snow -character",

    # LIYUE
    "liyue": "genshin impact liyue  landscape scenery -character",
    "the_chasm": "genshin impact the chasm underground landscape -character",

    # INAZUMA
    "inazuma": "genshin impact inazuma landscape scenery -character",
    "enkanomiya": "genshin impact enkanomiya landscape scenery -character",

    # SUMERU
    "sumeru": "genshin impact sumeru landscape -character",
    "sumeru_desert": "genshin impact sumeru desert landscape -character",

    # FONTAINE
    "fontaine": "genshin impact fontaine  landscape scenery -character",
    "fontaine_underwater": "genshin impact fontaine underwater scenery -character"
}

TARGET_IMAGES = 150

# Loop de execução
print("--- Iniciando Download dos Cenários de Genshin Impact ---")
#for label, keyword in classes.items():
#    download_images_generic(keyword, label, output_dir=OUTPUT_DIR, target_total=TARGET_IMAGES)

print("\nProcesso finalizado!")

# Carregando imagens do drive

### Treinamento e Validação

Dividiremos as imagens baixadas nas pastas `train` e `val`. Defina uma porcentagem.

In [ ]:
def split_dataset_3way(root_dir, train_ratio=0.7, val_ratio=0.15, seed=42):
    split_root = root_dir + "_split"
    train_dir = os.path.join(split_root, "train")
    val_dir   = os.path.join(split_root, "val")
    test_dir  = os.path.join(split_root, "test")

    # SE JÁ EXISTE, NÃO FAZ NADA (Segurança)
    if os.path.exists(split_root) and len(os.listdir(split_root)) > 0:
        print(f"Dataset já dividido em: {split_root}")
        return train_dir, val_dir, test_dir

    print("Iniciando divisão do dataset...")
    for dir_path in [train_dir, val_dir, test_dir]:
        os.makedirs(dir_path, exist_ok=True)

    random.seed(seed)
    # Lista classes ordenadas para garantir consistência
    classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])

    for class_name in classes:
        src_path = os.path.join(root_dir, class_name)
        images = sorted([f for f in os.listdir(src_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        random.shuffle(images)

        n_total = len(images)
        n_train = int(n_total * train_ratio)
        n_val   = int(n_total * val_ratio)

        # Criar subpastas
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Copiar arquivos
        for img in images[:n_train]:
            shutil.copy(os.path.join(src_path, img), os.path.join(train_dir, class_name, img))
        for img in images[n_train : n_train + n_val]:
            shutil.copy(os.path.join(src_path, img), os.path.join(val_dir, class_name, img))
        for img in images[n_train + n_val:]:
            shutil.copy(os.path.join(src_path, img), os.path.join(test_dir, class_name, img))

        print(f"  [{class_name}] {n_train} Train, {n_val} Val, {n_total - n_train - n_val} Test")

    return train_dir, val_dir, test_dir

In [ ]:
import os
import shutil
import random

# ==============================================================================
# 1. CONFIGURAÇÃO
# ==============================================================================
# Caminho da pasta
SOURCE_PATH = "genshin_landscape_dataset"

# Onde vamos salvar a versão organizada (treino/teste)
SPLIT_ROOT = "dataset_split_final"

# ==============================================================================
# 2. FUNÇÃO DE DIVISÃO (Caso a pasta venha "crua")
# ==============================================================================
def split_dataset_local(root_dir, target_root, train_ratio=0.7, val_ratio=0.15):
    # Cria pastas de destino
    for x in ['train', 'val', 'test']:
        os.makedirs(os.path.join(target_root, x), exist_ok=True)

    # Lista as classes (pastas dentro do dataset)
    classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    classes.sort()

    if not classes:
        print(f"❌ ERRO: Nenhuma pasta de classe encontrada dentro de {root_dir}")
        return

    print(f"Iniciando divisão do dataset...")

    for class_name in classes:
        class_dir = os.path.join(root_dir, class_name)
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp'))]

        images.sort()

        # Embaralha para garantir aleatoriedade
        random.shuffle(images)

        # Calcula quantidades
        n_total = len(images)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)

        # Define splits
        splits = {
            'train': images[:n_train],
            'val':   images[n_train:n_train+n_val],
            'test':  images[n_train+n_val:]
        }

        # Copia os arquivos
        for split_name, split_imgs in splits.items():
            dest_dir = os.path.join(target_root, split_name, class_name)
            os.makedirs(dest_dir, exist_ok=True)
            for img in split_imgs:
                shutil.copy(os.path.join(class_dir, img), os.path.join(dest_dir, img))

        print(f"  [{class_name}] Total: {n_total} -> Train: {len(splits['train'])} | Val: {len(splits['val'])} | Test: {len(splits['test'])}")

# ==============================================================================
# 3. LÓGICA PRINCIPAL
# ==============================================================================

if use_local:
  if not os.path.exists(SOURCE_PATH):
      print(f"❌ A pasta '{SOURCE_PATH}' não foi encontrada!")
  else:
      # Verifica se a pasta já tem 'train' e 'val' dentro (se já veio pronta)
      contents = os.listdir(SOURCE_PATH)
      if 'train' in contents and 'val' in contents:
          print("✅ Estrutura pronta detectada (pastas train/val já existem).")
          TRAIN_DIR = os.path.join(SOURCE_PATH, 'train')
          VAL_DIR = os.path.join(SOURCE_PATH, 'val')
          TEST_DIR = os.path.join(SOURCE_PATH, 'test') if 'test' in contents else VAL_DIR
      else:
          print("⚠️ Estrutura 'crua' detectada. Realizando divisão automática...")
          # Limpa destino anterior se existir
          if os.path.exists(SPLIT_ROOT): shutil.rmtree(SPLIT_ROOT)

          # Executa a divisão
          split_dataset_local(SOURCE_PATH, SPLIT_ROOT)

          TRAIN_DIR = os.path.join(SPLIT_ROOT, 'train')
          VAL_DIR = os.path.join(SPLIT_ROOT, 'val')
          TEST_DIR = os.path.join(SPLIT_ROOT, 'test')

      # Confirmação final
      print("\n--- Caminhos Configurados ---")
      print(f"Train: {TRAIN_DIR}")
      print(f"Val:   {VAL_DIR}")
      print(f"Test:  {TEST_DIR}")
else:
  drive.mount('/content/drive')
  TRAIN_DIR, VAL_DIR, TEST_DIR = split_dataset_3way(OUTPUT_DIR)


## Dataset

Implemente um Dataset PyTorch que carregue as imagens baixadas com suas respectivas classes. Aplique data augmentation e carregue em batches.

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [ ]:
# Augmentation para TREINO (Variações para aprender melhor)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # Zoom aleatório
    transforms.RandomHorizontalFlip(p=0.5),              # Espelhamento Horizontal
    transforms.RandomRotation(degrees=15),               # Rotação leve
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02), # Hue baixo!
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Para VALIDAÇÃO e TESTE (Apenas redimensionar e normalizar)
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset   = datasets.ImageFolder(VAL_DIR,   transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(TEST_DIR,  transform=val_test_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")
print("Total treino:", len(train_dataset))
print("Total validação:", len(val_dataset))

images, labels = next(iter(train_loader))
print("Batch shape:", images.shape)
print("Labels:", labels[:10])



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## Definição do Modelo

Defina aqui o modelo que será utilizado, sendo implementação própria ou um modelo pré-treinado. Teste diversas arquiteturas diferentes e verifique qual delas tem melhor desempenho em validação.

In [ ]:
import torchvision.models as models
import torch.nn as nn # Adicionar esta linha para importar o módulo nn

NUM_CLASSES = 10


# ----------------------------------------------------------
# B — MobileNetV2 (transfer learning — rápido e eficiente)
# ----------------------------------------------------------
def build_mobilenet(num_classes):
    model = models.mobilenet_v2(weights="IMAGENET1K_V1")

    for param in model.features.parameters():
      param.requires_grad = False

    model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    return model


# ----------------------------------------------------------
# D — EfficientNet-B0 (ótima acurácia com baixo custo)
# ----------------------------------------------------------
def build_efficientnet(num_classes, dropout_rate=0.2):
    model = models.efficientnet_b0(weights="IMAGENET1K_V1")

    for param in model.features.parameters():
      param.requires_grad = False

    for param in model.features[-3:].parameters():
        param.requires_grad = True

    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout_rate, inplace=True),
        nn.Linear(in_features, num_classes),
    )

    return model

# ----------------------------------------------------------
# Função auxiliar para escolher o modelo
# ----------------------------------------------------------
def get_model(name, num_classes, dropout_rate=0.2):
    name = name.lower()
    if name == "mobilenet":
        return build_mobilenet(num_classes)
    elif name == "efficientnet":
        return build_efficientnet(num_classes, dropout_rate)
    else:
        raise ValueError("Model name inválido.")

## Treinamento

Defina a função de custo e o otimizador do modelo. Em seguida, implemente o código de treinamento e treine-o. Ao final, exiba as curvas de treinamento e validação para a loss e a acurácia.

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        """
        Args:
            patience (int): Quantas épocas esperar sem melhoria antes de parar.
            min_delta (float): Mínima mudança para considerar como melhoria.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict() # Salva o estado inicial
        elif val_loss > self.best_loss - self.min_delta:
            # Não melhorou o suficiente
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} de {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            # Melhorou - Resetar contador e salvar novo melhor
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0

    def load_best_weights(self, model):
        # Restaura os pesos da melhor época encontrada
        model.load_state_dict(self.best_model_state)

In [ ]:

import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.optim as optim # Adicionar esta linha para importar o otimizador

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------------------------------------------
# Função de treino + validação
# ---------------------------------------------------------

def train_model(model, train_loader, val_loader, epochs=20, lr=1e-4, save_name="best_model.pth", patience=3, use_early_stopping=False, use_scheduler=False):

    early_stopping = None
    if use_early_stopping:
        early_stopping = EarlyStopping(patience=patience, min_delta=1e-4)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    scheduler = None
    if use_scheduler:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    lrs = []

    best_val_acc = 0.0
    last_lr = lr

    for epoch in range(epochs):
        print(f"\nEPOCH {epoch+1}/{epochs}")
        model.train()

        epoch_loss = 0
        correct = 0
        total = 0

        # -------------------- TREINO --------------------
        pbar = tqdm(train_loader, desc="Training", leave=False)
        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss = epoch_loss / len(train_loader)

        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # -------------------- VALIDAÇÃO --------------------
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)

                outputs = model(imgs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        val_loss = val_loss / len(val_loader)

        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        if scheduler:
            scheduler.step(val_loss)
            current_lr = optimizer.param_groups[0]['lr']
            lrs.append(current_lr)
            if current_lr != last_lr:
                print(f"Learning Rate diminuiu: {last_lr:.1e} -> {current_lr:.1e}")
                last_lr = current_lr

        if early_stopping:
            early_stopping(val_loss, model)
            if early_stopping.early_stop:
                print("Stopping early. Model has stopped improving.")
                break

        # ---------------- Salvar melhor modelo ----------------
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_name)
            print(f"Modelo salvo com Acc: {best_val_acc:.4f}")

    # salvar checkpoint final
    print("-" * 30)
    if best_val_acc > 0:
      print(f"Carregando o melhor modelo do disco (Acc: {best_val_acc:.4f})...")
      model.load_state_dict(torch.load(save_name))
    else:
      print("\nO modelo não melhorou em nenhuma época")

    return train_losses, val_losses, train_accs, val_accs, lrs

In [ ]:
def train_and_evaluate(model_name, model, train_loader, val_loader, epochs=20, lr=1e-4, patience=3, use_early_stopping=False, use_scheduler=False):

    print(f"\n==============================")
    print(f" Treinando modelo: {model_name}")
    print(f"==============================")

    filename = f"best_model_{model_name}.pth"

    train_losses, val_losses, train_accs, val_accs, lrs = train_model(
        model,
        train_loader,
        val_loader,
        epochs=epochs,
        lr=lr,
        save_name=filename,
        patience=patience,
        use_early_stopping=use_early_stopping,
        use_scheduler=use_scheduler
    )

    best_val_acc = max(val_accs)

    return {
        "name": model_name,
        "train_losses": train_losses,
        "val_losses": val_losses,
        "train_accs": train_accs,
        "val_accs": val_accs,
        "best_val_acc": best_val_acc,
        "lrs": lrs
    }


In [ ]:
models_to_test = {
    #"SimpleCNN": get_model("cnn"),
    #"MobileNetV2": get_model("mobilenet", num_classes=len(train_dataset.classes))
    #"ResNet50": get_model("resnet50")
    "EfficientNet": get_model("efficientnet", num_classes=len(train_dataset.classes), dropout_rate=0.4)
}


In [ ]:

results = []


for name, model in models_to_test.items():
    model = model.to(device)
    r = train_and_evaluate(name, model, train_loader,val_loader, epochs = 50, lr=1e-4, patience=8, use_early_stopping=True, use_scheduler=True)
    results.append(r)


In [ ]:
print("\n===== RESULTADO FINAL =====\n")
for r in results:
    print(f"{r['name']:20s}  |  Best Val Acc: {r['best_val_acc']:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

def plot_comparison(results):
# Define o estilo visual "Clean"
    sns.set_style("whitegrid")

    # Cores Profissionais
    c_train = '#2980b9' # Azul Belize
    c_val   = '#c0392b' # Vermelho Pomegranate
    c_text  = '#2c3e50' # Cinza Escuro (quase preto)
    c_lr    = '#8e44ad' # Roxo Wisteria (para o LR)

    for res in results:
        train_loss = res['train_losses']
        val_loss = res['val_losses']
        train_acc = res['train_accs']
        val_acc = res['val_accs']

        lrs = res.get('lrs')

        epochs = range(1, len(train_loss) + 1)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        # --- GRÁFICO 1: LOSS ---
        # Área de divergência (sombra leve onde Validação > Treino)
        ax1.fill_between(epochs, train_loss, val_loss, color=c_val, alpha=0.05)

        ax1.plot(epochs, train_loss, color=c_train, linestyle='--', linewidth=2, label='Treino', marker='o', markersize=4)
        ax1.plot(epochs, val_loss, color=c_val, linewidth=2.5, label='Validação', marker='s', markersize=4)

        ax1.set_title("Evolução da Perda (Loss)", fontsize=14, fontweight='bold', color=c_text, pad=15)
        ax1.set_xlabel("Épocas", fontsize=11)
        ax1.set_ylabel("Loss", fontsize=11)
        ax1.legend(frameon=True, facecolor='white', framealpha=0.9, loc='upper right')

        # Remove bordas desnecessárias (topo e direita)
        sns.despine(ax=ax1)

        # Seta indicando o MELHOR momento (Menor Loss)
        min_loss = min(val_loss)
        idx_best = val_loss.index(min_loss)
        best_epoch_loss = idx_best + 1
        val_at_best = min_loss
        train_at_best = train_loss[idx_best]

        y_span = max(max(train_loss), max(val_loss)) - min(min(train_loss), min(val_loss))
        offset = y_span * 0.15
        marker_gap = y_span * 0.025 # Distância da PONTA DA SETA (o "respiro")

        # Flags para controlar limites
        text_goes_down = False

        if train_at_best > val_at_best:
            # Texto EMBAIXO, Seta aponta pra CIMA
            # xy (ponta da seta) fica um pouco abaixo do valor real (min_loss - gap)
            xy_point = (best_epoch_loss, val_at_best - marker_gap)
            xy_text_pos = (best_epoch_loss, val_at_best - offset)
            text_goes_down = True
        else:
            # Texto EM CIMA, Seta aponta pra BAIXO
            # xy (ponta da seta) fica um pouco acima do valor real (min_loss + gap)
            xy_point = (best_epoch_loss, val_at_best + marker_gap)
            xy_text_pos = (best_epoch_loss, val_at_best + offset)

        ax1.annotate(f'Melhor: {min_loss:.3f}', xy=xy_point, xytext=xy_text_pos,
                     arrowprops=dict(facecolor=c_text, shrink=0.0, width=2, headwidth=8),
                     color=c_text, ha='center', fontweight='bold',
                     bbox=dict(boxstyle="round,pad=0.3", fc="white", ec=c_text, alpha=0.8))

        # --- SOLUÇÃO 2: EVITAR QUE O TEXTO SAIA DO GRÁFICO EMBAIXO ---
        if text_goes_down:
            ymin, ymax = ax1.get_ylim()
            if xy_text_pos[1] < ymin:
                ax1.set_ylim(bottom=xy_text_pos[1] - (y_span * 0.05))

        # --- GRÁFICO 2: ACCURACY ---
        ax2.plot(epochs, train_acc, color=c_train, linestyle='--', linewidth=2, label='Treino', marker='o', markersize=4)
        ax2.plot(epochs, val_acc, color=c_val, linewidth=2.5, label='Validação', marker='s', markersize=4)

        ax2.set_title("Evolução da Acurácia", fontsize=14, fontweight='bold', color=c_text, pad=15)
        ax2.set_xlabel("Épocas", fontsize=11)
        ax2.set_ylabel("Acurácia", fontsize=11)
        ax2.legend(frameon=True, facecolor='white', framealpha=0.9, loc='lower right')

        sns.despine(ax=ax2)

        # Seta indicando o PICO de performance
        max_acc = max(val_acc)
        best_epoch_acc = val_acc.index(max_acc) + 1

        # Span da Accuracy para calcular gap proporcional
        acc_span = max(val_acc) - min(val_acc) if max(val_acc) != min(val_acc) else 0.1
        acc_offset = acc_span * 0.15
        acc_gap = acc_span * 0.025

        ax2.annotate(f'Pico: {max_acc*100:.1f}%', xy=(best_epoch_acc, max_acc - acc_gap), xytext=(best_epoch_acc, max_acc - acc_offset),
                     arrowprops=dict(facecolor=c_text, shrink=0.0, width=2, headwidth=8),
                     color=c_text, ha='center', fontweight='bold')

        plt.tight_layout()
        plt.show()

    # =========================================================
    # NOVO CÓDIGO: GRÁFICO EXTRA DE LR (Eixo Duplo)
    # =========================================================
    if lrs is not None and len(lrs) > 0:
      with plt.style.context('seaborn-v0_8'):
        # Cria a figura e o primeiro eixo (Loss)
        fig, ax_loss = plt.subplots(figsize=(10, 5))

        # Título e Eixo X
        ax_loss.set_title(f"Impacto da Learning Rate na Loss ({res.get('name', 'Modelo')})", fontsize=12)
        ax_loss.set_xlabel("Epoch")

        # Eixo da Esquerda (Loss - Azul)
        ax_loss.set_ylabel("Train Loss", color="#1f77b4", fontweight='bold') # Usei o azul padrão do seaborn
        l1 = ax_loss.plot(epochs, train_loss, color="#1f77b4", linewidth=2.5, label="Train Loss") # Removi o 'o' (marker) para um look mais clean
        ax_loss.tick_params(axis="y", labelcolor="#1f77b4")
        ax_loss.grid(True, alpha=0.5, linestyle='--') # Grade mais suave

        # Eixo da Direita (LR - Vermelho)
        ax_lr = ax_loss.twinx() # Cria o eixo duplo
        ax_lr.set_ylabel("Learning Rate", color="#d62728", fontweight='bold') # Usei o vermelho padrão do seaborn
        l2 = ax_lr.plot(epochs, lrs, marker="", linestyle="-.", color="#d62728", linewidth=1.5, label="Learning Rate") # Mudei o marker e linestyle
        ax_lr.tick_params(axis="y", labelcolor="#d62728")


        ax_lr.set_yscale('log')

        ax_lr.yaxis.set_major_formatter(
            FuncFormatter(lambda y, _: f'{y:.0e}')
        )



        # Legenda Unificada
        lines = l1 + l2
        labels = [l.get_label() for l in lines]
        ax_loss.legend(lines, labels,
                    loc='upper center',
                    bbox_to_anchor=(0.5, 1.15),
                    ncol=2,
                    frameon=False,
                    fontsize=10) # Ajustei o tamanho da fonte da legenda

        fig.tight_layout()
        plt.show()

        print("-" * 80) # Linha separadora entre modelos

plot_comparison(results)

## Inferência

Calcule algumas métricas como acurácia, matriz de confusão, etc. Em seguida, teste o modelo em novas imagens das classes correspondentes mas de outras fontes (outro buscador, fotos próprias, etc).

In [ ]:
# Mapeamento: Sub-região visual -> Nação
region_hierarchy = {
    "mondstadt": "Mondstadt", "dragonspine": "Mondstadt (Dragonspine)",
    "liyue": "Liyue",
    "inazuma": "Inazuma", "enkanomiya": "Inazuma (Enkanomiya)",
    "sumeru": "Sumeru", "sumeru_desert": "Sumeru (Deserto)",
    "fontaine": "Fontaine", "fontaine_underwater": "Fontaine (Debaixo d'água)"
}

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import torch

def evaluate_complete(model, loader, class_names):
    # --- 1. Obter Predições ---
    model.eval()
    all_preds = []
    all_labels = []

    print("Calculando métricas no Test Set...")

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- Relatório em Texto  ---
    print("\n--- Relatório de Classificação ---")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # --- 2. Preparar Dados ---
    # Métricas
    report_dict = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    acc_score = report_dict['accuracy']

    df_report = pd.DataFrame(report_dict).transpose()
    classes_df = df_report.drop(['accuracy', 'macro avg', 'weighted avg'])

    # Matriz
    cm = confusion_matrix(all_labels, all_preds)

    # Configuração de fonte
    TITLE_SIZE = 18    # Título principal
    LABEL_SIZE = 18    # Nomes das regiões e eixos
    ANNOT_SIZE = 18    # Números dentro do heatmap/barras

    # ==========================================
    # FIGURA 1: DASHBOARD DE MÉTRICAS (Heatmap + Barras)
    # ==========================================
    fig, axes = plt.subplots(1, 2, figsize=(16, 9), gridspec_kw={'width_ratios': [2, 1]})

    # --- Plot 1: Heatmap ---
    sns.heatmap(classes_df[['precision', 'recall', 'f1-score']],
                annot=True, cmap='RdYlGn', fmt='.2f', vmin=0.7, vmax=1.0,
                cbar=False, ax=axes[0],
                annot_kws={'size': ANNOT_SIZE}) # <--- Aumenta números internos

    axes[0].set_title(f'Métricas por classe (Acurácia global: {acc_score:.2%})', fontsize=TITLE_SIZE, pad=15)

    # Aumentar fontes dos eixos
    axes[0].tick_params(axis='y', labelsize=LABEL_SIZE, rotation=0) # <--- Nomes das Regiões
    axes[0].tick_params(axis='x', labelsize=LABEL_SIZE)             # <--- Precision/Recall...

    # --- Plot 2: Barras de Support ---
    sns.barplot(x=classes_df['support'], y=classes_df.index, ax=axes[1], color='skyblue', edgecolor='black')

    axes[1].set_title('Qtd. Imagens (Support)', fontsize=TITLE_SIZE, pad=15)
    axes[1].set_ylabel('')
    axes[1].set_yticklabels([])
    axes[1].tick_params(axis='x', labelsize=LABEL_SIZE)
    axes[1].grid(axis='x', linestyle='--', alpha=0.6)

    # Números na ponta das barras
    for i, v in enumerate(classes_df['support']):
        axes[1].text(v + 0.5, i, f"{int(v)}", color='black', va='center', fontweight='bold', fontsize=ANNOT_SIZE)


    plt.tight_layout()
    plt.show()

    # ==========================================
    # FIGURA 2: MATRIZ DE CONFUSÃO
    # ==========================================
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={'size': ANNOT_SIZE})

    plt.title('Matriz de Confusão', fontsize=TITLE_SIZE, pad=20)
    plt.xlabel('Predito', fontsize=16)
    plt.ylabel('Real', fontsize=16)
    plt.xticks(rotation=90, fontsize=16)
    plt.yticks(rotation=0, fontsize=16)

    plt.tight_layout()
    plt.show() # Mostra o segundo gráfico


    # ==============================================================================
    # FIGURA 3: PRECISÃO POR REGIÃO (Com cores temáticas)
    # ==============================================================================
    # Definição das Cores Temáticas
    genshin_colors = {
        # --- Regras Específicas (Sub-regiões visuais distintas) ---
        'Desert':      '#E6CC80',  # Cor de Areia (Para Sumeru Desert)
        'Underwater':  '#00CED1',  # Turquesa Escuro (Para Fontaine Underwater)
        'Dragonspine': '#B0E0E6',  # Azul Gelo Pálido
        'Enkanomiya':  '#191970',  # Azul Meia-Noite (Bem escuro)

        # --- Regras Gerais (Nações) ---
        'Mondstadt':   '#4CC2A5',  # Anemo (Verde Água)
        'Liyue':       '#FFCA3D',  # Geo (Amarelo Ouro)
        'Inazuma':     '#9E77CB',  # Electro (Roxo)
        'Sumeru':      '#74C25C',  # Dendro (Verde Folha - Pega o que sobrou de Sumeru)
        'Fontaine':    '#4899E8',  # Hydro (Azul Claro - Pega o que sobrou de Fontaine)
        'Natlan':      '#FF4500',  # Pyro (Laranja Avermelhado)
    }

    # 2. Lógica de Atribuição (Busca por Palavra-Chave)
    bar_colors = []
    for region_name in classes_df.index:
        color_found = '#CCCCCC' # Cor padrão (Cinza) caso não ache nada

        # O loop percorre o dicionário na ordem definida acima
        for key, color in genshin_colors.items():
            if key.lower() in region_name.lower():
                color_found = color
                break # <--- ACHOU A COR ESPECÍFICA? PARA DE PROCURAR.

        bar_colors.append(color_found)

    plt.figure(figsize=(12, 6))

    # 3. Plotar SEM ordenar (usa a ordem original do classes_df)
    sns.barplot(x=classes_df['precision'], y=classes_df.index, palette=bar_colors)

    plt.title('Precisão por Região', fontsize=TITLE_SIZE, pad=15)
    plt.xlabel('Precisão (0 a 1)', fontsize=14)
    plt.xlim(0, 1.15)
    plt.grid(axis='x', linestyle='--', alpha=0.3)

    # 4. Colocar a porcentagem na frente de cada barra
    for i, (val, name) in enumerate(zip(classes_df['precision'], classes_df.index)):
        # Escolhe a cor do texto (preto ou a mesma cor da barra para ficar estiloso)
        txt_color = '#333333'
        plt.text(val + 0.01, i, f"{val:.1%}", va='center', fontsize=12, fontweight='bold', color=txt_color)

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_grid(model, dataset, num_images=9, seed=None):
    model.eval()

    if seed is not None:
        g = torch.Generator()
        g.manual_seed(seed)
        indices = torch.randperm(len(dataset), generator=g)[:num_images].tolist()
    else:
        indices = random.sample(range(len(dataset)), num_images)

    plt.figure(figsize=(12, 12))

    for i, idx in enumerate(indices):
        img, label = dataset[idx]
        input_tensor = img.unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            prob = torch.nn.functional.softmax(output, dim=1)
            conf, pred_idx = torch.max(prob, 1)

        pred_class = dataset.classes[pred_idx.item()]
        real_class = dataset.classes[label]

        # Mapear para Nação
        nation = region_hierarchy.get(pred_class, "")

        # Cor: Verde se acertou, Vermelho se errou
        color = 'green' if pred_class == real_class else 'red'

        # Desnormalizar para exibir (assumindo ImageNet stats)
        inv_norm = transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
        img_show = inv_norm(img).permute(1, 2, 0).cpu().numpy()
        img_show = np.clip(img_show, 0, 1)

        plt.subplot(3, 3, i + 1)
        plt.imshow(img_show)
        plt.axis('off')
        plt.title(f"Pred: {pred_class}\nReal: {real_class}\n{nation}\n{conf.item():.1%}",
                  color=color, fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_uncertainty(model, dataset, num_images=5, seed=None):
    model.eval()

    # 1. Lógica de Sorteio (IGUAL AO GRID)
    if seed is not None:
        g = torch.Generator()
        g.manual_seed(seed)
        indices = torch.randperm(len(dataset), generator=g)[:num_images].tolist()
    else:
        indices = random.sample(range(len(dataset)), num_images)

    # Parâmetros de denormalização
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    print(f"--- Visualização de Incerteza (Amostras: {indices}) ---")

    for i, idx in enumerate(indices):
        # Pega a imagem direto do dataset pelo índice sorteado
        img_tensor, label = dataset[idx]

        # Prepara o batch de 1 imagem
        input_tensor = img_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)

        # --- Plotagem ---
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        # Imagem
        img_show = img_tensor.cpu().numpy().transpose((1, 2, 0))
        img_show = std * img_show + mean
        img_show = np.clip(img_show, 0, 1)

        real_class = dataset.classes[label]
        ax1.imshow(img_show)
        ax1.axis("off")
        ax1.set_title(f"Real: {real_class}\n(Índice {idx})")

        # Gráfico de Barras
        p = probs[0].cpu().numpy()
        predicted_idx = np.argmax(p)
        pred_class = dataset.classes[predicted_idx]

        # Cores
        colors = ["lightgray"] * len(dataset.classes)
        colors[label] = "green" # Correto
        if predicted_idx != label:
            colors[predicted_idx] = "red" # Erro

        y_pos = np.arange(len(dataset.classes))
        ax2.barh(y_pos, p, align='center', color=colors)
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(dataset.classes)
        ax2.invert_yaxis()
        ax2.set_xlabel('Probabilidade')
        ax2.set_title(f"Predição: {pred_class}\n({p[predicted_idx]:.1%} certeza)")
        ax2.set_xlim(0, 1.1)

        plt.tight_layout()
        plt.show()

In [ ]:
# 1. Carregar o Melhor Modelo
last_model_name = list(models_to_test.keys())[-1]
print(f"🔄 Carregando pesos do modelo: {last_model_name}...")

model_to_eval = models_to_test[last_model_name]
model_to_eval.load_state_dict(torch.load(f"best_model_{last_model_name}.pth"))
model_to_eval.to(device)

In [ ]:
# 2. Executar Métricas Gerais
print("\n=== AVALIAÇÃO GERAL ===")
evaluate_complete(model_to_eval, test_loader, train_dataset.classes)

In [ ]:
# 3. Executar Visualização em Grid
print("\n=== EXEMPLOS VISUAIS ===")
visualize_grid(model_to_eval, test_dataset, num_images=6, seed=42)

In [ ]:
# --- NOVA CHAMADA ---
# Note que agora passamos apenas o 'test_dataset', não mais o 'loader'
print("\n=== ANÁLISE DE INCERTEZA (Amostras Aleatórias) ===")
visualize_uncertainty(model_to_eval, test_dataset, num_images=3, seed=42)

In [ ]:
def predict_genshin(image_path, real_label=None):
    # Carregar e transformar
    try:
        img = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Erro ao abrir imagem: {e}")
        return

    # Usa a mesma transformação de validação/teste
    img_t = val_test_transforms(img).unsqueeze(0).to(device)

    model_to_eval.eval()
    with torch.no_grad():
        outputs = model_to_eval(img_t)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        conf, idx = torch.max(probs, 1)

    pred_class = train_dataset.classes[idx.item()]
    pred_nation = region_hierarchy.get(pred_class, "Desconhecido")

    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis('off')

    title_text = f"Predito: {pred_class.upper()} ({pred_nation})"
    color = 'black'

    if real_label:
        title_text += f"\nReal: {real_label.upper()}"
        color = 'green' if real_label == pred_class else 'red'

    title_text += f"\nConfiança: {conf.item():.1%}"
    plt.title(title_text, color=color, fontweight='bold')
    plt.show()

def test_random_local_image():
    if not os.path.exists(TEST_DIR):
        print("Pasta de teste não encontrada.")
        return

    # Sorteia classe e imagem
    random_class = random.choice(os.listdir(TEST_DIR))
    class_path = os.path.join(TEST_DIR, random_class)

    try:
        random_image = random.choice(os.listdir(class_path))
        full_path = os.path.join(class_path, random_image)
        print(f"Testando imagem local: {random_image}")
        predict_genshin(full_path, real_label=random_class)
    except IndexError:
        print(f"A pasta {random_class} parece vazia.")

In [ ]:
def predict_from_root():
    root_folder = "/content"
    # Extensões válidas
    valid_extensions = ('.jpg', '.jpeg', '.png', '.webp', '.bmp')

    print(f"--- Varrendo apenas a raiz ({root_folder}) ---")

    files_found = 0

    # Lista tudo na pasta raiz
    for filename in os.listdir(root_folder):
        full_path = os.path.join(root_folder, filename)

        # A MÁGICA ESTÁ AQUI:
        # 1. Verifica se é arquivo (ignora pastas!)
        # 2. Verifica se tem extensão de imagem
        if os.path.isfile(full_path) and filename.lower().endswith(valid_extensions):

            # Filtro extra: Ignorar arquivos de sistema do Colab (opcional)
            if filename == "best_model.pth": continue

            print(f"\n📸 Encontrado: {filename}")
            try:
                # Chama a função de predição que já criamos
                predict_genshin(full_path)
                files_found += 1
            except Exception as e:
                print(f"Erro ao ler arquivo: {e}")

    if files_found == 0:
        print(">> Nenhuma imagem solta encontrada na raiz!")
        print(">> Arraste os arquivos para a área lateral esquerda (fora de pastas).")

# Executar
predict_from_root()

In [ ]:
import os

root_folder = "/content"

print(f"--- Deletando arquivos .png começando com '2025' na raiz ({root_folder}) ---")

deleted_count = 0
for filename in os.listdir(root_folder):
    if filename.startswith('2025') and filename.lower().endswith('.png') and os.path.isfile(os.path.join(root_folder, filename)):
        full_path = os.path.join(root_folder, filename)
        try:
            os.remove(full_path)
            print(f"🗑️ Deletado: {filename}")
            deleted_count += 1
        except Exception as e:
            print(f"Erro ao deletar {filename}: {e}")

if deleted_count == 0:
    print(">> Nenhuma imagem .png começando com '2025' foi encontrada para deletar.")
else:
    print(f"\nTotal de arquivos deletados: {deleted_count}")